# Расчет степени корректности распознавания цифр

**Задача**

Требуется посчитать:
- % правильно распознанных ячеек last4Numbers (то есть если поле last4Numbers полностью совпадает, все 4 цифры)
- % правильно распознанных цифр для колонки last4Numbers (то есть среди всех всех цифр)
- % правильно распознанных ячеек momentCard (то есть если поле momentCard полностью совпадает, все 8 цифр)
- общий % правильно распознанных цифр для колонки momentCard (то есть среди всех всех цифр)

dcopen_predicts.csv - распознавание модели
dcopen_validation.csv - фактические значения

Интересуют строки только с pageSide==BACK (тк цифры есть только на обратной стороне заявления)

In [1]:
import pandas as pd

In [2]:
data_val = pd.read_csv('dcopen_validation.csv', sep = ';')
data_pred = pd.read_csv('dcopen_predicts.csv')

## Предварительная обработка и подготовка данных

In [3]:
data_val.shape

(136, 12)

In [4]:
data_pred.shape

(135, 13)

In [5]:
df = data_val
df.drop(df.columns[[1, 2, 3, 4, 6, 7, 9, 10, 11]], axis = 1, inplace = True)

In [6]:
df = df.rename(columns = {'last4Numbers':'l4n_val', 'momentCard': 'mc_val'})

In [7]:
df['l4n_pred'] = data_pred['last4Numbers']
df['mc_pred'] = data_pred['momentCard']

In [8]:
df = df.reindex(columns=['pdf_name_i', 'l4n_pred', 'l4n_val', 'mc_pred', 'mc_val'])

In [9]:
df['pdf_pred'] = data_pred['pdf_name_i']

Определим, есть ли несовпадения в строках по имени png-файлов

In [10]:
df.query('pdf_name_i != pdf_pred')

,pdf_name_i,l4n_pred,l4n_val,mc_pred,mc_val,pdf_pred
135,ADJ5QN_APCAC_110433711_2.png,NaN,6067.0,NaN,14526231.0,NaN


Таких не оказалось (есть только один отсутствующий - последний). Подтверждено совпадение порядков данных в обеих таблицах.

In [11]:
df = df.dropna()

In [12]:
df.head(5)

,pdf_name_i,l4n_pred,l4n_val,mc_pred,mc_val,pdf_pred
1,ADHY7A_APCAC_110378177_2.png,4667.0,4667.0,74554043.0,14554043.0,ADHY7A_APCAC_110378177_2.png
3,ADHYN3_APCAC_110380025_2.png,3216.0,5633.0,11453467.0,14433067.0,ADHYN3_APCAC_110380025_2.png
5,ADHYXW_APCAC_110406316_2.png,4040.0,1248.0,44078861.0,14676264.0,ADHYXW_APCAC_110406316_2.png
7,ADHZ4L_APCAC_110447988_2.png,831.0,1831.0,14433944.0,14432444.0,ADHZ4L_APCAC_110447988_2.png
11,ADHZ8M_APCAC_110418645_2.png,7332.0,7357.0,74530247.0,14552249.0,ADHZ8M_APCAC_110418645_2.png


In [13]:
df_l4n = df.drop(['mc_pred', 'mc_val', 'pdf_pred'], axis = 1)
df_l4n = df_l4n.rename(columns = {'l4n_val':'validated', 'l4n_pred': 'predicts'})

In [14]:
df_l4n['predicts'] = df_l4n['predicts'].astype('int').astype('str')
df_l4n['validated'] = df_l4n['validated'].astype('int').astype('str')
df_l4n = df_l4n.reset_index()
df_l4n.head(10)

,index,pdf_name_i,predicts,validated
0,1,ADHY7A_APCAC_110378177_2.png,4667,4667
1,3,ADHYN3_APCAC_110380025_2.png,3216,5633
2,5,ADHYXW_APCAC_110406316_2.png,4040,1248
3,7,ADHZ4L_APCAC_110447988_2.png,831,1831
4,11,ADHZ8M_APCAC_110418645_2.png,7332,7357
5,13,ADHZ9K_APCAC_110432995_2.png,3452,5827
6,15,ADHZB3_APCAC_110398898_2.png,6778,6778
7,17,ADHZBM_APCAC_110389516_2.png,7833,1833
8,19,ADHZDD_APCAC_110382354_2.png,3558,9358
9,21,ADHZE6_APCAC_110437131_2.png,9813,9843


In [15]:
df_mc = df.drop(['l4n_pred', 'l4n_val', 'pdf_pred'], axis = 1)
df_mc = df_mc.rename(columns = {'mc_val':'validated', 'mc_pred': 'predicts'})

In [16]:
df_mc['predicts'] = df_mc['predicts'].astype('int').astype('str')
df_mc['validated'] = df_mc['validated'].astype('int').astype('str')
df_mc = df_mc.reset_index()
df_mc.head(10)

,index,pdf_name_i,predicts,validated
0,1,ADHY7A_APCAC_110378177_2.png,74554043,14554043
1,3,ADHYN3_APCAC_110380025_2.png,11453467,14433067
2,5,ADHYXW_APCAC_110406316_2.png,44078861,14676264
3,7,ADHZ4L_APCAC_110447988_2.png,14433944,14432444
4,11,ADHZ8M_APCAC_110418645_2.png,74530247,14552249
5,13,ADHZ9K_APCAC_110432995_2.png,14536132,14554018
6,15,ADHZB3_APCAC_110398898_2.png,14605153,14605157
7,17,ADHZBM_APCAC_110389516_2.png,22071465,12814465
8,19,ADHZDD_APCAC_110382354_2.png,2630280,14491221
9,21,ADHZE6_APCAC_110437131_2.png,14429741,14428511


### Функции

In [17]:
# Удаление строк, где количество проверяемых символов не совпадает
# На входе df с двумя обязательными колонками (validated и predicts в формате 'str')
def delete_bad_rows (df):
    to_delete_row = []
    print('Случаи несовпадения длины проверяемых записей (будут исключены из дальнейшего сравнения):', '\n')
    for row in range(len(df)):
        if len(list(df['predicts'][row])) != len(list(df['validated'][row])):
            print(df.loc[row], '\n')
            to_delete_row.append(row)
    # print(to_delete_row)
    df = df.drop(to_delete_row).reset_index(drop=True)
    return df

In [18]:
# Функция для расчета сходства по ЦИФРОВЫМ символам
# На входе df с двумя обязательными колонками (validated и predicts в формате 'str')
def matrix(df):
    cross = pd.DataFrame(index=range(10),columns=range(10)).fillna(0)
    for item in range(len(df)):
        symbol_val = list(df['validated'][item])
        symbol_pred = list(df['predicts'][item])
        for position in range(len(symbol_val)):
            row = int(symbol_val[position])
            column = int(symbol_pred[position])
            cross.iloc[row, column] += 1
    return cross

In [19]:
def resume(cross):
    total = cross.sum().sum()

    correct_pos = 0
    for index in range(len(cross)):
        correct_pos += cross[index][index]

    print(f'Всего символов: {total}\nИз них правильно определено: {correct_pos}')
    print(f'Доля правильных: {correct_pos/total:.2%}', '\n')
    print(cross)

## momentCard

In [20]:
correct = df_mc[df_mc['predicts'] == df_mc['validated']]['validated'].count()
print(f'Всего записей: {len(df_mc)}\nИз них правильно определено: {correct}')
print(f'Доля правильных записей: {correct/len(df_mc):.2%}')

Всего записей: 62
Из них правильно определено: 3
Доля правильных записей: 4.84%


In [21]:
df_mc = delete_bad_rows (df_mc)

Случаи несовпадения длины проверяемых записей (будут исключены из дальнейшего сравнения): 

index                                   19
pdf_name_i    ADHZDD_APCAC_110382354_2.png
predicts                           2630280
validated                         14491221
Name: 8, dtype: object 

index                                   49
pdf_name_i    ADJ186_APCAC_110425596_2.png
predicts                            255997
validated                         14236591
Name: 24, dtype: object 

index                                   89
pdf_name_i    ADJ2T5_APCAC_110446996_2.png
predicts                           4227653
validated                         14227603
Name: 41, dtype: object 



In [22]:
cross = matrix(df_mc)
resume(cross)

Всего символов: 472
Из них правильно определено: 299
Доля правильных: 63.35% 

    0   1   2   3   4   5   6   7   8   9
0  19   2   1   0   2   2   5   2   1   2
1   4  55   3   4  13   0   0  18   0   0
2   3   0  25   2   0   1   1   4   1   0
3   0   0   0  29   0   3   0   1   0   0
4   1  10   1   1  75   1   5   1   0   2
5   0   1   0   3   0  35   0   2   0   3
6   4   3   3   3   1   0  17   0   6   4
7   0   1   5   4   0   3   0  18   2   0
8   2   1   3   1   1   0   0   0  14   3
9   2   3   0   3   1   3   0   3   2  12


## last4Numbers

In [23]:
correct = df_l4n[df_l4n['predicts'] == df_l4n['validated']]['validated'].count()
print(f'Всего записей: {len(df_l4n)}\nИз них правильно определено: {correct}')
print(f'Доля правильных записей: {correct/len(df_l4n):.2%}')

Всего записей: 62
Из них правильно определено: 13
Доля правильных записей: 20.97%


In [24]:
df_l4n = delete_bad_rows (df_l4n)

Случаи несовпадения длины проверяемых записей (будут исключены из дальнейшего сравнения): 

index                                    7
pdf_name_i    ADHZ4L_APCAC_110447988_2.png
predicts                               831
validated                             1831
Name: 3, dtype: object 

index                                   27
pdf_name_i    ADHZQ0_APCAC_110374517_2.png
predicts                              5884
validated                              881
Name: 12, dtype: object 

index                                   34
pdf_name_i    ADJ0BS_APCAC_110398364_2.png
predicts                              3753
validated                              965
Name: 16, dtype: object 

index                                   51
pdf_name_i    ADJ1A0_APCAC_110382341_2.png
predicts                              6160
validated                              106
Name: 25, dtype: object 

index                                   59
pdf_name_i    ADJ1TT_APCAC_110409234_2.png
predicts                       

In [25]:
cross = matrix(df_l4n)
resume(cross)

Всего символов: 213
Из них правильно определено: 135
Доля правильных: 63.38% 

    0   1   2   3   4   5   6  7   8   9
0  11   0   1   0   0   1   1  0   0   1
1   0  16   0   0   4   0   0  2   0   0
2   2   2  10   0   0   1   0  3   1   0
3   1   1   2  14   0   2   1  1   0   0
4   1   2   1   1  18   0   0  1   0   0
5   0   0   0   4   0  17   0  1   0   0
6   0   0   2   2   0   2  11  1   5   1
7   0   0   3   1   1   1   0  9   2   0
8   1   0   0   0   1   1   1  0  16   2
9   2   0   0   5   1   4   0  0   1  13
